# Tubular Products

The Tubular Products Division of Babcock and Wilcox manufactures steel tubing in a variety of sizes and for many different uses.  Currently, three mills handle production. The firm wishes to ascertain how a fourth mill of a different configuration will affect the (optimal) distribution of work (and associated costs) among the mills.

The Gams code below gives data for existing mills 1 to 3 and two designs for a new mill 4 and 5, versus an array of 16 products.  The products comprise all combinations of standard  or high-pressure tubing: 1/2, 1, 2, or 8 inch diameters; and thick or thin tube walls.  Missing values indicate products that cannot be manufactured at the given mill.  The weekly demands and working capacities are also given.

### Which mill (if any) should be built?  

Hint: While this could be modeled as a mixed integer programming model, it is required here instead to set up a single linear programming model, and solve that model three (or more) times to answer the question at hand.

In [1]:
# Load the gams extension
%load_ext gams_magic

__The GAMS Data:__

In [2]:
%%gams
sets
tube            /standard,high/,
size            /half,one,two,eight/,
wall            /thick,thin/,
p(tube,size,wall) "products",
m "mills"         /1*5/;
set I /"none", "4", "5"/;

p(tube,size,wall) = yes;

table c(tube,size,wall,m) "cost (in dollars) per 1000 pounds of p at m"
                        1    2    3    4    5
standard.half.thick    90   75   70   63   64
standard.half.thin     80   70   65   60   64
standard.one.thick    104   85   83   77   64
standard.one.thin      98   79   80   74   64
standard.two.thick    123  101  110   99   90
standard.two.thin     113   94  100   84  100
standard.eight.thick       160  156  140  160
standard.eight.thin        142  150  130  140
high.half.thick       140  110       122  110
high.half.thin        124   96       101  101
high.one.thick        160  133       138  134
high.one.thin         143  127       133  139
high.two.thick        202  150       160  150
high.two.thin         190  141       140  160
high.eight.thick           190       220  250
high.eight.thin            175       200  200;

table t(tube,size,wall,m) "time (in hours) per 1000 pounds of p at m"
                         1    2    3    4    5
standard.half.thick    0.8  0.7  0.5  0.6  0.6
standard.half.thin     0.8  0.7  0.5  0.6  0.6
standard.one.thick     0.8  0.7  0.5  0.6  0.6
standard.one.thin      0.8  0.7  0.5  0.6  0.6
standard.two.thick     0.8  0.7  0.5  0.6  0.6
standard.two.thin      0.8  0.7  0.5  0.6  0.6
standard.eight.thick        0.9  0.5  0.6  0.6
standard.eight.thin         0.9  0.5  0.6  0.6
high.half.thick        1.5  0.9       1.2  1.2
high.half.thin         1.5  0.9       1.2  1.2
high.one.thick         1.5  0.9       1.2  1.2
high.one.thin          1.5  0.9       1.2  1.2
high.two.thick         1.5  0.9       1.2  1.2
high.two.thin          1.5  0.9       1.2  1.2
high.eight.thick            1.0       1.5  1.5
high.eight.thin             1.0       1.5  1.5;

parameter d(tube,size,wall) "weekly demand in 1000 pounds for product"/
standard.half.thick  100, 
standard.half.thin   630, 
standard.one.thick   500, 
standard.one.thin    980, 
standard.two.thick   720, 
standard.two.thin    240, 
standard.eight.thick 75, 
standard.eight.thin  22,
high.half.thick      50, 
high.half.thin       22, 
high.one.thick       353, 
high.one.thin        55, 
high.two.thick       125, 
high.two.thin        35, 
high.eight.thick     100, 
high.eight.thin      10/;

parameter b(m) "mill m capacity (in hours)"
/1 800, 2 480, 3 1280, 4 960, 5 960/;

In [3]:
%%gams

positive variable f(tube,size,wall,m);
free variable cost "cost of building mills";

equations demandRequirement(tube,size,wall),maxCapacityHours(m),objfunc;

* EQUATION (MODEL) DEFINITION 
demandRequirement(tube,size,wall)..
  sum(m,f(tube,size,wall,m)) =g= d(tube,size,wall);

maxCapacityHours(m)..
  sum((tube,size,wall),t(tube,size,wall,m)*f(tube,size,wall,m)) =l= b(m);

* Objective function
objfunc..
  cost =e= sum((tube,size,wall,m),c(tube,size,wall,m)*f(tube,size,wall,m));
    
* Variable bound
f.up(tube,size,wall,m)$(c(tube,size,wall,m) = 0) = 0;
  
model tubprods /all/;


In [4]:
%%gams
* solve for the case when only mills 1,2,3 are built
f.fx(p,'4') = 0;
f.fx(p,'5') = 0;
solve tubprods using lp min cost;

parameter soln(I);
soln("none") = cost.l;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),InfeasibleGlobal (4),407094.0909,22,81,LP,CPLEX,0.027


In [5]:
%%gams

* solve for the case when mills 1,2,3,4 are built
f.fx(p,'5') = 0;
f.up(p,'4') = inf;
solve tubprods using lp min cost;
soln("4") = cost.l;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),378899.1111,22,81,LP,CPLEX,0.004


In [6]:
%%gams

* solve for the case when mills 1,2,3,5 are built
f.fx(p,'4') = 0;
f.up(p,'5') = inf;
solve tubprods using lp min cost;
soln("5") = cost.l;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),367626.0,22,81,LP,CPLEX,0.012


__Pull the objective values from GAMS and determine which mill to build:__

In [7]:
# solution should hold all the LP solution values you want to compare
%gams_pull -d soln
soln.set_index('I',inplace=True)
print('Build mill ', soln['value'].idxmin())

Build mill  5
